# Test

In [ ]:
import pandas as pd
import csv

def load_transaction_csv(file_path,header=True,articles="Articles",separator=","):
    if header:
        transactions = pd.read_csv(file_path)
    else :
        transactions = pd.read_csv(file_path,header=None)
    if len(transactions.columns) == 1:
        articles = transactions.columns[0]
    if separator:
        transactions = transactions[transactions[articles].apply(lambda x: isinstance(x, str))]
        transactions[articles] = transactions[articles].apply(lambda x: [i.strip() for i in x.split(",")])
    

    return transactions
def load_transactions(file_path,simple_case=True,header=True,separator=","):
    " Give pandas dataset of transaction "
    if file_path[-4:] == ".csv":
        print("yes")
        return load_transaction_csv(file_path,header=header)

transactions = load_transactions('../data/stupid.csv',simple_case=False,header=True)
display(transactions)

yes


,Transaction,Articles
0,T1,"[pain, lait]"
1,T2,"[pain, beurre]"
2,T3,"[lait, beurre, confiture]"
3,T4,"[pain, lait, beurre]"
4,T5,"[lait, beurre]"


In [5]:
articles = transactions["Articles"]
display(articles)

0                 [pain, lait]
1               [pain, beurre]
2    [lait, beurre, confiture]
3         [pain, lait, beurre]
4               [lait, beurre]
Name: Articles, dtype: object

In [9]:
all_motifs = sorted(set(item for sublist in articles for item in sublist))
print(all_motifs)

['beurre', 'confiture', 'lait', 'pain']


In [17]:
df = transactions.copy()
df_exploded = df.explode('Articles')
df_encoded = pd.get_dummies(df_exploded, columns=['Articles'])
df_encoded = df_encoded.groupby(df_encoded.index).sum()

In [18]:
display(df_encoded)

,Transaction,Articles_beurre,Articles_confiture,Articles_lait,Articles_pain
0,T1T1,0,0,1,1
1,T2T2,1,0,0,1
2,T3T3T3,1,1,1,0
3,T4T4T4,1,0,1,1
4,T5T5,1,0,1,0


In [53]:
class TransactionDf():

    def __init__(self,file_path,header=False,target_column=None,separator=","):
        self.file_paths = [file_path]
        self.headers = [header]
        self.target_columns = [target_column]
        self.separators = [separator]
        self.dfs = []
        self.load_transactions(file_path,header,target_column,separator)
    def load_transaction_csv(self,file_path,header,target_column,sep):
        if header:
            transactions = pd.read_csv(file_path)
        else :
            transactions = pd.read_csv(file_path,header=None)
        if len(transactions.columns) == 1:
            target_column = transactions.columns[0]
        elif target_column not in transactions.columns:
            raise ValueError(f"the targeted column {target_column} is not in {transactions.columns}")

        if sep:
            transactions = transactions[transactions[target_column].apply(lambda x: isinstance(x, str))]
            transactions[target_column] = transactions[target_column].apply(lambda x: [i.strip() for i in x.split(",")])
        

        # all this hot encode a column and keep all the others column
        transactions_exploded = transactions[target_column].explode()

        transactions_dummies = pd.get_dummies(transactions_exploded)


        transactions_dummies = transactions_dummies.groupby(transactions_dummies.index).sum()

        transactions = transactions.drop(columns=[target_column]).join(transactions_dummies)
        return transactions
    


    def load_transactions(self,file_path,header=True,target_column=None,sep=","):
        " Give pandas dataset of transaction "
        res = None
        if file_path[-4:] == ".csv":
            print("yes")
            try :
                res = self.load_transaction_csv(file_path,header,target_column,sep)
            except ValueError as e:
                print(f"Error: {e}")
        
        
        if res is not None:
            self.dfs.append(res)
    
    def displays(self):
        for df in self.dfs:
            display(df)
    def combine(self,indexes=[],column_to_check=None):
        if len(self.dfs) == 2:
            indexes = [0,1]

        print(indexes)
        res = None
        #df_combined = pd.concat([self.dfs[indexes[0]], self.dfs[indexes[1]]], axis=1).fillna(0).astype(int)
        if column_to_check is not None:
            # Merge using outer join to keep all names
            df_merged = pd.merge(self.dfs[indexes[0]], self.dfs[indexes[1]], on=column_to_check, how='outer', suffixes=('_1', '_2'))

            df_merged = df_merged.fillna(0)
            numeric_cols = [c for c in df_merged.columns if c != column_to_check]
            df_merged[numeric_cols] = df_merged[numeric_cols].astype(int)

            # Combine columns with similar meaning
            all_columns = set(self.dfs[indexes[0]].columns).union(self.dfs[indexes[1]].columns) - {column_to_check}

            for col in all_columns:
                # Find all columns corresponding to this original column (with _1/_2 suffixes or exact match)
                cols_to_sum = [c for c in df_merged.columns if c == col or c.startswith(col + "_")]
                if cols_to_sum:
                    df_merged[col] = df_merged[cols_to_sum].sum(axis=1)
                    df_merged.drop(columns=cols_to_sum, inplace=True, errors='ignore')
            res = df_merged
            res = res[sorted(res.columns)]

        display(res)

        

In [54]:
T_df = TransactionDf('../data/stupid.csv',header=True,target_column="Articles")
T_df.displays()

yes


,Transaction,beurre,confiture,lait,pain
0,T1,0,0,1,1
1,T2,1,0,0,1
2,T3,1,1,1,0
3,T4,1,0,1,1
4,T5,1,0,1,0


In [55]:
T_df.load_transactions('../data/stupid.csv',header=True,target_column="Articles")
T_df.displays()
T_df.combine(column_to_check="Transaction")

yes


,Transaction,beurre,confiture,lait,pain
0,T1,0,0,1,1
1,T2,1,0,0,1
2,T3,1,1,1,0
3,T4,1,0,1,1
4,T5,1,0,1,0


,Transaction,beurre,confiture,lait,pain
0,T1,0,0,1,1
1,T2,1,0,0,1
2,T3,1,1,1,0
3,T4,1,0,1,1
4,T5,1,0,1,0


[0, 1]


,Transaction,beurre,confiture,lait,pain
0,T1,0,0,2,2
1,T2,2,0,0,2
2,T3,2,2,2,0
3,T4,2,0,2,2
4,T5,2,0,2,0
